In [ ]:
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import tensorflow as tf
from datasets import Dataset
import tqdm as notebook_tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

df_test = pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')
df_test = df_test

In [ ]:
df_test.drop('label', axis=1, inplace=True)
df_test

,gender,profession,ideology_binary,ideology_multiclass,tweet
0,male,journalist,right,moderate_right,"Ayer, en Guadalajara: feministas arrojaban áci..."
1,male,journalist,right,moderate_right,"“Chile Vamos”, o sea, la centro-derechita coba..."
2,male,journalist,right,moderate_right,"Se está tratando de silenciar que en Bolivia, ..."
3,male,journalist,right,moderate_right,"@user Lucía, lo siento pero no es así. Hebe ja..."
4,male,journalist,right,moderate_right,JxC no dio quórum para eliminar el impuesto a ...
...,...,...,...,...,...
43755,male,politician,right,right,"No, si ahora resulta que todo el mundo es prim..."
43756,male,politician,right,right,Tenemos un buen futuro si entre todos nos pone...
43757,male,politician,right,right,Cuando el [POLITICAL_PARTY] dice la estupidez ...
43758,male,politician,right,right,La entrevista de @user a @user hoy en El Mundo...


In [ ]:

# Initialize the tokenizer
checkpoint_name= 'juan-glez29/ideology_ft'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/483k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/283k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test['ideology_multiclass'] = label_encoder.fit_transform(df_test['ideology_multiclass'])

In [ ]:
def tokenize_function(example):
    return tokenizer(example['tweet'], truncation=True)

# Conversión del DataFrame de pandas a un Dataset de Hugging Face para pasarlo al modelo
huggingface_dataset = Dataset.from_pandas(df_test)
huggingface_dataset

Dataset({
    features: ['gender', 'profession', 'ideology_binary', 'ideology_multiclass', 'tweet'],
    num_rows: 43760
})

In [ ]:
# Aplicamos la tokenización a todo el dataset
tokenized_dataset = huggingface_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["tweet","gender","profession","ideology_binary"])
test_dataset = tokenized_dataset.rename_column("ideology_multiclass", "labels")

Map:   0%|          | 0/43760 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 43760
})

In [ ]:
# Define test trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_trainer = Trainer(model, data_collator=data_collator, tokenizer=tokenizer)

predicted_results = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = predicted_results.predictions.argmax(-1) # Get the highest probability prediction

# Calculate accuracy
labels = test_dataset["labels"]
print(f"Accuracy:\t{accuracy_score(y_true=labels, y_pred=y_pred)}")

precision, recall, f1, _ = precision_recall_fscore_support(y_true=labels, y_pred=y_pred, average='weighted') # o weighted si estan desbalceadas las clases
print(f"Precision:\t{precision}")
print(f"Recall:\t\t{recall}")
print(f"F1-Score:\t{f1}")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Accuracy:	0.46723034734917734
Precision:	0.4638423125902883
Recall:		0.46723034734917734
F1-Score:	0.45049395615150883


In [ ]:
# # Tokenize text inputs
# tokenized_inputs = tokenizer(df_test['tweet'][100], return_tensors="pt")

# # Pass tokenized inputs through the Transformer model
# outputs = model(**tokenized_inputs)

# # Get predictions
# labels = ['izq', 'izq_mod', 'dcha_mod', 'dcha']
# prediction = torch.argmax(outputs.logits)

# print(outputs)

# print(f"The prediction is {labels[prediction]}")